<h1> ______________________________________ </h1>

<h1> Fiber barrel simulation analysis </h1>
<h1> ______________________________________ </h1>

<p style="font-size: 17px; color: black;"> In this Notebook we analyse the results from photons generated in 10000 MC events simulated in a Fiber barrel with photosensors in one end and Aluminium in the other end. </p>


<p style="font-size: 17px; color: black;"> For this, we first run some test to check that the MC output is indeed the type of events we wanted to simulate, later we calculate the system efficiency. </p>



<h1> ____________ </h1>
<h2> This version </h2>
<h1> ____________ </h1>

-  <p style="font-size: 17px; color: black;"> 10K events. </p>
-  <p style="font-size: 17px; color: black;"> Using perfect photosensors. </p>


-  <p style="font-size: 17px; color: black;"> Geant4 fundamental units: </p>

 -  <p style="font-size: 17px; color: black;"> Length [L]: mm (milimeter) </p>
 -  <p style="font-size: 17px; color: black;"> Time [T]: ns (nanosecond) </p>
 -  <p style="font-size: 17px; color: black;"> Energy [E]: MeV (megaelectronvolt) </p>



In [1]:
import numpy              as np
import pandas             as pd

In [2]:
import scipy              
from scipy                import stats, integrate
from scipy.signal         import convolve

In [3]:
import matplotlib.pyplot  as plt

from matplotlib.ticker    import FormatStrFormatter # to set the format of the plot's axis
from matplotlib.patches   import Rectangle # To add blanck spaces in tabular legends

In [4]:
import os
import math
import tables             as tb

<h1> _________________________________________________________________________________________________________ </h1>


<h1> __________________________ </h1>
<h2> Functions </h2>

-  <p style="font-size: 17px; color: black;"> <b> gauss(x, a,  mu, sigma): </b> Gaussian normalized to $a$. </p>


-  <p style="font-size: 17px; color: black;"> <b> gauss_sum(x, a0,  mu0, sigma0, a1, mu1, sigma1): </b> Sum of 2 gaussians. </p>


-  <p style="font-size: 17px; color: black;"> <b> dirac(x, x0): </b> Dirac delta at $x0$. </p>




In [5]:
# Fitting distributions to define

# note: pdf are normilized to 1 so we have yo add an amplitude param
def gauss(x, a,  mu, sigma):
    
    gaussian = stats.norm.pdf(np.sort(x), mu, sigma) 
    
    return (a/gaussian.sum())*gaussian

def gauss_sum(x, a0,  mu0, sigma0, a1, mu1, sigma1):
    return a0*stats.norm.pdf(x, mu0, sigma0)  + a1*stats.norm.pdf(x, mu1, sigma1)

def dirac(x, x0):
    return np.where((x - x0) == 0, 1, 0)


<h1> __________________________ </h1>
<h2> Global parameters </h2>

-  <p style="font-size: 17px; color: black;"> <b> Pandas dataset params: </b> Allows to set the max number of columns and rows that are shown in a pandas dataset. </p>

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.min_rows', 100)

-  <p style="font-size: 17px; color: black;"> <b> Plots params: </b> In case you want to fix the parameters for all plots. </p>

In [7]:
# plt.rcParams["figure.figsize"] = 10, 8
# plt.rcParams["font.size"] = 10

-  <p style="font-size: 17px; color: black;"> <b> Analysis parameters </b> </p>

 -  <p style="font-size: 17px; color: black;"> <b> patata: </b> patata. </p>
 


In [8]:
dt = 1e3 # [ns] = 1us


<h1> __________________________ </h1>
<h2> Data </h2>

-  <p style="font-size: 17px; color: black;"> <b> Reading the file </b> </p>

-  <p style="font-size: 17px; color: black;"> File's path </p>

In [9]:
path = '/home/investigator/mariandbt/nexus/'

# filename = os.path.join(path, "Fiber_barrel_meth_20230718.next.h5")
filename = os.path.join(path, "Fiber_barrel_meth.next.h5")

-  <p style="font-size: 17px; color: black;"> In a .h5 file there are several objects, so we print them all to then pick the one we're interested in </p>

In [10]:
with tb.open_file(filename) as file:
    print(file)

/home/investigator/mariandbt/nexus/Fiber_barrel_meth.next.h5 (File) ''
Last modif.: '2023-07-19T08:39:08+00:00'
Object Tree: 
/ (RootGroup) ''
/MC (Group) ''
/MC/configuration (Table(44,)) ''
/MC/hits (Table(0,)) ''
/MC/particles (Table(25433,)) ''
/MC/sns_positions (Table(31,)) ''
/MC/sns_response (Table(2914,)) ''



-  <p style="font-size: 17px; color: black;"> Read the file and make a copy to work with, this way it's easier to re-start the copy instead of re-reading the file. </p>

In [11]:
# %%timeit # WITH THIS YOU'RE ONLY TIMING, BUT IT DOESN'T REALLY RUN THE COMMANDS YOU TIME
data = pd.read_hdf(filename, "/MC/particles")


In [12]:
dst = data.copy()

In [13]:
dst

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,initial_volume,final_volume,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
0,0,1,opticalphoton,1,0,-1.404256,-2.217978,-1.778009,0.000000,3.664290,2.618024,5.879508,0.034619,LAB,B2_WLS_3,3.418565e-06,3.261716e-06,5.164748e-06,3.453869e-06,5.897119e-06,1.514854e-06,7.000000e-06,10.378572,none,OpWLS
1,1,1,opticalphoton,1,0,2.765507,0.039092,-0.023025,0.000000,-0.134909,-4.507802,-4.614694,0.023627,LAB,B2_WLS_23,-2.866395e-06,-4.493553e-06,-4.537815e-06,-1.703293e-06,-6.757489e-06,-6.596444e-07,7.000000e-06,7.083087,none,OpWLS
2,1,2,opticalphoton,0,1,-0.134909,-4.507802,-4.614694,2.423627,-0.145957,-5.362825,-4.253528,2.431968,B2_WLS_23,B2,-6.583679e-07,1.166341e-06,2.991944e-06,2.828724e-06,-1.159644e-06,1.182826e-06,3.278038e-06,1.584331,OpWLS,OpWLS
3,1,3,opticalphoton,0,2,-0.145957,-5.362825,-4.253528,10.931969,-0.234106,-5.416022,3.668533,11.013484,B2,B2,1.032052e-06,1.201353e-07,-2.808908e-06,9.515089e-07,-4.173660e-07,2.808908e-06,2.994917e-06,15.369582,OpWLS,OpWLS
4,1,4,opticalphoton,0,3,-0.234106,-5.416022,3.668533,19.513483,0.290125,1.966498,7.500000,19.840174,B2,TEFLON_CAPS_1,-2.231148e-06,-1.391456e-06,1.206162e-06,-6.424664e-07,-5.084460e-07,2.774476e-06,2.892922e-06,81.342567,OpWLS,Transportation
5,2,1,opticalphoton,1,0,-4.560319,0.825753,-6.598651,0.000000,-4.574703,0.806311,-6.679331,0.000281,LAB,B2_WLS_15,-1.195259e-06,-1.615820e-06,-6.705258e-06,-2.661316e-06,-1.984786e-06,-6.162631e-06,7.000000e-06,0.084227,none,OpWLS
6,2,2,opticalphoton,0,1,-4.574703,0.806311,-6.679331,2.400281,-4.677353,0.661242,-6.602036,2.401289,B2_WLS_15,B2,4.649296e-07,2.942815e-06,1.856911e-06,-1.891059e-06,-2.621535e-06,1.369629e-06,3.510618e-06,0.193858,OpWLS,OpWLS
7,2,3,opticalphoton,0,2,-4.677353,0.661242,-6.602036,10.901289,1.460039,-4.329438,7.700000,11.114061,B2,F_SENSOR_SENSAREA,-8.838175e-08,6.696492e-07,9.024141e-08,4.309673e-08,-9.111385e-08,6.739629e-07,6.814579e-07,53.020496,OpWLS,Transportation
8,3,1,opticalphoton,1,0,-0.144849,-4.318639,-0.391905,0.000000,-2.607489,-3.742336,1.331268,0.010208,LAB,B2_WLS_20,-5.632754e-06,1.318165e-06,3.941386e-06,-5.926058e-06,2.369451e-06,2.875333e-06,7.000000e-06,3.060400,none,OpWLS
9,3,2,opticalphoton,0,1,-2.607489,-3.742336,1.331268,2.410208,-4.641346,-2.325520,-2.440707,2.540572,B2_WLS_20,B2,3.005799e-06,2.885464e-07,6.362125e-08,1.053019e-06,1.275616e-06,2.527072e-06,3.020287e-06,32.331188,OpWLS,OpWLS


In [14]:
np.shape(dst), len(dst)

((25433, 25), 25433)

In [15]:
print(len(dst.loc[dst['final_volume'] == 'F_SENSOR_SENSAREA']))
print(len(dst.loc[(dst['final_volume'].str.contains('SENSAREA')) & (dst['initial_volume'].str.contains('B2'))]))

3052
3052


In [16]:
dst.loc[(dst['final_volume'].str.contains('SENSOR')) & (dst['initial_volume'].str.contains('B2'))]

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,initial_volume,final_volume,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
7,2,3,opticalphoton,0,2,-4.677353,0.661242,-6.602036,10.901289,1.460039,-4.329438,7.700000,11.114061,B2,F_SENSOR_SENSAREA,-8.838175e-08,6.696492e-07,9.024141e-08,4.309673e-08,-9.111385e-08,6.739629e-07,6.814579e-07,53.020496,OpWLS,Transportation
10,3,4,opticalphoton,0,2,-4.641346,-2.325520,-2.440707,11.040571,3.601221,3.855603,7.700000,11.720721,B2,F_SENSOR_SENSAREA,4.673447e-07,-6.254555e-07,6.086437e-07,-4.957274e-07,6.031375e-07,6.087138e-07,9.899762e-07,151.917053,OpWLS,Transportation
20,7,4,opticalphoton,0,3,4.488218,-1.584506,3.912082,13.320701,-4.672851,-0.106974,7.603064,14.764960,B2,F_SENSOR_CASE_16,-1.171970e-06,1.340977e-06,1.133178e-06,-3.416554e-08,1.371830e-06,1.603976e-06,2.110882e-06,336.352051,OpWLS,Transportation
24,9,3,opticalphoton,0,2,-2.573237,4.755649,0.875310,10.903829,-2.454591,4.127542,7.700000,10.941739,B2,F_SENSOR_SENSAREA,6.665749e-07,4.992017e-07,2.674168e-06,-6.028903e-07,5.744977e-07,2.674168e-06,2.800839e-06,7.147964,OpWLS,Transportation
30,12,2,opticalphoton,0,1,-2.045127,4.014291,3.687045,2.417679,3.990108,3.040494,7.700000,2.839702,B2_WLS_10,F_SENSOR_SENSAREA,-7.883091e-07,1.089247e-07,-1.205345e-06,5.795031e-07,-4.182349e-07,1.255152e-06,1.444351e-06,95.678551,OpWLS,Transportation
33,13,4,opticalphoton,0,3,4.595277,0.988948,2.846307,10.921366,-2.634889,4.739177,7.609857,11.333781,B2,F_SENSOR_CASE_10,2.045498e-06,-1.740864e-06,5.422252e-07,-1.670941e-06,-2.275070e-07,2.159832e-06,2.740197e-06,96.679085,OpWLS,Transportation
43,15,4,opticalphoton,0,3,-3.065840,-3.808763,2.315795,13.322514,1.112015,-4.664425,7.700000,13.708342,B2,F_SENSOR_SENSAREA,-2.229084e-06,-2.042389e-07,1.287869e-06,1.179471e-06,1.025618e-06,2.055746e-06,2.582467e-06,88.920311,OpWLS,Transportation
47,16,4,opticalphoton,0,3,-4.651927,0.603938,4.957635,10.918056,-5.167444,0.244604,7.700000,10.975402,B2,F_SENSOR_SENSAREA,1.959250e-06,1.416781e-06,-1.650948e-06,6.938718e-08,-2.133147e-06,2.004104e-06,2.927723e-06,11.264866,OpWLS,Transportation
49,16,6,opticalphoton,0,2,-4.662092,0.807781,4.776597,10.916977,2.179015,-4.455954,7.548461,11.068849,B2,F_SENSOR_CASE_25,6.413417e-07,-2.815874e-06,-1.864040e-07,1.973362e-06,1.376033e-06,1.608597e-06,2.893996e-06,32.360409,OpWLS,Transportation
53,17,6,opticalphoton,0,5,-0.683178,-4.853105,-4.330623,19.411072,-0.700487,-4.927481,7.700000,19.514107,B2,F_SENSOR_SENSAREA,1.056618e-06,-1.200494e-08,-2.558287e-06,3.309322e-07,2.714293e-07,2.734635e-06,2.767927e-06,19.636833,OpWLS,Transportation


In [17]:
dst['length'].mean()

27.54335594177246